In [1]:
using JuMP
using Ipopt

In [2]:
# scales: tons, years, MXNia, hours, trips
n1 = -22.239 # ML, slope
n2 = 49.811 # ML, intersect
l1 = -0.0028 # q, slope
l2 = 0.1667 # q, intersect
a1 = 1/3.4E7 # proportion of migrating squid, where 3.4E7 max(e^(tau-b1))
K = 1208770 # carrying capacity in t
f = 40 # l of fuel per trip
B_h = 7.203 # hours per fisher
B_f = 2 # fisher per panga

### Initial values #############################################################
m0 = 1.4 # population growth rate
g0 = 5492603.58 # cost per unit of transport all boats, MXN/trip

tau0 = 42. # temperature
q0 = 0.01 # squid catchability
y_S0 = 0.5 # proportion of migrated squid
R_tt0 = 0.5 # trader cooperation
S0 = 1208770 # size of the squid population
c_t0 = m0*g0 # fleet cost of transport
E0 = 1. # fishing effort
C0 = 120877 # squid catch
p_e0 = 164706 # max p_e comtrade
p_f0 = 15438 # max p_f datamares
R0 = C0*p_f0-(c_t0+E0)

1.858409479988e9

In [3]:
using DataFrames
import XLSX
df1 = dropmissing(DataFrame(XLSX.readtable("./DATA/R3_data.xlsx", "Sheet1", infer_eltypes=true)...))
# load columns
y = Int64.(df1[:year]) #
pe = Float64.(df1[:pe_MXNiat]) #
pf = Float64.(df1[:pf_MXNiat]) #
ct = Float64.(df1[:C_t]) #
ssh = Float64.(df1[:essh_avg]) #
ml = Float64.(df1[:ML]) #
ys = map(x->parse(Float64,x),df1[:y_S]) #
### New max time
tmax = length(y)

13

In [4]:
model = JuMP.Model(solver = IpoptSolver(print_level=3, max_iter=99900,print_frequency_iter=50,sb="yes"));

- We're dumping the `E`:`Escal` dynamic, removing `h1`,`h2` and just constraining `E`; where the constraint on `E` was previously the condition for `Escal`.
    - This may need some work though. `E` should be between [0,1], therefore `h1`,`h2` seem to be manual constraints anyhow... This means the model should probably be run twice to identify this first, then to grab `C` etc.
- It's probable that `m` and `g` are not variable and should indeed be set as input conditions.
- Implementing `flag == 1` model for the moment to include `p_min` as a parameter.
- We can probably add in migration costs rather than fixed costs (`c_t`).
- `a1` seems to be conditional on `max(tau)`...
- `cachability` is too hardcoded for my liking


In [5]:
@variable(model, 38.750 <= b1 <= 42.1, start = 41.750) # isotherm depth
@variable(model, -6.9 <= b2 <= -3.987, start = -5.696) # isotherm depth
@variable(model, 11.478 <= b3 <= 16.4, start = 16.397) # isotherm depth
@variable(model, 0.01 <= beta <= 0.1, start = 0.0736) # slope of demand-price function
@variable(model, 1000.0 <= c_p <= 2148, start = 1776.25) # cost of processing, MXNia/t
@variable(model, 0.0 <= g <= 2.9, start = g0) # population growth rate
@variable(model, 20000.0 <= gamma <= 51000, start = 49200) # maximum demand, t
@variable(model, 2368793.0 <= m <= 8450159, start = m0) # cost per unit of transport all boats, MXN/trip
@variable(model, 11956952.0 <= w_m <= 28108539, start = 13355164) # min wage per hour all fleet

@variable(model, tau[t=1:tmax]) # temperature
@variable(model, p_e[t=1:tmax]) # export price
@variable(model, q[t=1:tmax]) # catchability squid population
@variable(model, ML[t=1:tmax]) # mantle length
@variable(model, y_S[t=1:tmax]) # distance of squid migration from initial fishing grounds
@variable(model, R_tt[t=1:tmax]) # trader cooperation
@variable(model, c_t[t=1:tmax]) # cost of transport
@variable(model, p_min[t=1:tmax]) # minimum wage
@variable(model, R[t=1:tmax]) # revenue of fishers
# c_t is per trip so we need to upscale E hr > fisher > trip
@variable(model, -3.1e09 <= E[t=1:tmax] <= 1.6e9) # fishing effort
@variable(model, S[t=1:tmax]) # size of the squid population
@variable(model, C[t=1:tmax]) # squid catch
@variable(model, p_f[t=1:tmax]) # price for fishers

@variable(model, data[t=1:tmax])
@variable(model, match)

@constraint(model, [t=1:tmax], tau[t] == b1+b2*cos.(t)+b3*sin.(t));
@NLconstraint(model, [t=1:tmax], p_e[t] == gamma*(C[t])^(-beta));
@NLconstraint(model, [t=1:tmax-1], E[t+1] == E[t]+p_f[t]*q[t]*E[t]*S[t]-c_t[t]*(E[t]/(B_h+B_f)));
@constraint(model, [t=1:tmax], R_tt[t] == 1-y_S[t]); 
@constraint(model, [t=1:tmax-1], S[t+1] == S[t]+g*S[t]*(1-(S[t]/K))-q[t]*E[t]*S[t]); 
# I decided to use fixed costs over migration, that equally well/better predicted
# catches over m* (y_S[t]); (source: LabNotesSquid, April 11)
@constraint(model, [t=1:tmax], c_t[t] == m*f); 
@NLconstraint(model, [t=1:tmax], C[t] == q[t]*E[t]*S[t]);
@NLconstraint(model, [t=1:tmax], p_min[t] == (E[t]*w_m)/C[t]);
@constraint(model, [t=1:tmax], p_f[t] == (p_e[t]-c_p)*(1-R_tt[t])+R_tt[t]*p_min[t]);
@constraint(model, [t=1:tmax-1], R[t+1] == C[t+1]*p_f[t+1]-c_t[t]*(E[t]/(B_h+B_f)));

LoadError: [91mCannot multiply a quadratic expression by a variable[39m

In [6]:
# Input data
function migration(t)
    x = ys[t] == 1.0 ? a1*exp.(tau[t]-b1) : ys[t];
    # Coerce squid migration to [0,1]
    if x < 0.0
        0.0
    elseif x > 1.0
        1.0
    else
        x
    end
end
@constraint(model, [t=1:tmax], y_S[t] == migration(t));
catchability(t) = ml[t] == 1 ? l1*tau[t]+l2 : 0.0018*ml[t]-0.0318;
@constraint(model, [t=1:tmax], q[t] == catchability(t));
@constraint(model, ML[t=1:tmax] .== ml[t]);
@constraint(model, data[t=1:tmax] .== pf[t]);

# Initial conditions
@constraint(model, tau[1] == tau0);  # temperature
@constraint(model, p_f[1] == p_f0); # max p_f datamares
@constraint(model, p_e[1] == p_e0); # max p_e comtrade
@constraint(model, y_S[1] == y_S0); # proportion of migrated squid
@constraint(model, R_tt[1] == R_tt0); # trader cooperation
@constraint(model, c_t[1] == c_t0); # fleet cost of transport
@constraint(model, R[1] == R0); # revenue of fishers
@constraint(model, C[1] == C0); # squid catch
@constraint(model, q[1] == q0); # squid catchability
@constraint(model, E[1] == E0); # fishing effort
@constraint(model, S[1] == S0); # size of the squid population

#Minimise ther differences between our model and the data
#NOTE: Fig1 only currently
@NLconstraint(model, match == sum(abs(p_f[t] - data[t]) for t in 1:tmax));
@NLobjective(model, Min, match);

In [7]:
solve(model)

:Error

Total number of variables............................:      192
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       22
                     variables with only upper bounds:        0
Total number of equality constraints.................:      115
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0


Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 1
Number of equality constraint evaluations            = 1
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations           